In [1]:
## Check relay turn on and turn off
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import pytz
from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

def turn_on_turn_off(relayNumber):
        # check whether the relay 1 or 2 is on Auto mode then turn on and turn off automatically
        db = DatabaseManager()
        results = db.read_relaymode_temp(relayNumber) # setting relay number
                
        current_datetime = datetime.now() 
        current_datetime = current_datetime.strftime('%Y-%m-%dT%H:00:00')

        buffer_datetime = datetime.now() + timedelta(hours=1)
        buffer_datetime = buffer_datetime.strftime('%Y-%m-%dT%H:00:00')

        print(current_datetime)
        print(buffer_datetime)

        conn = sqlite3.connect("/home/pi/smart_plug/database/pythonsqlite.db")
        query = f"SELECT * from automaterelay where relaynumber={relayNumber} and start_timestamp BETWEEN '{current_datetime}' and '{buffer_datetime}'"
       
       # query = f"SELECT * from automaterelay where relaynumber={relayNumber} and start_timestamp BETWEEN '2023-12-07T22:00:00' and '2023-12-07T23:00:00'"
        print(query)
        
        new_dataframe = pd.read_sql_query(query, conn)

        print(new_dataframe)


        if len(results) > 0: # check whether relay is on Auto Mode
                # check if new_dataframe is empty then turn off the relay
                if(new_dataframe.empty):
                     # Turn OFF
                    print(f"Relay {relayNumber} turned off in AutoMode")

                current_time = datetime.now()
                # Check if current time is within any interval
                for index, row in new_dataframe.iterrows():
                    startdatetime = pd.to_datetime(row['start_timestamp'])
                    enddatetime = pd.to_datetime(row['end_timestamp'])
                    print("startdatetime", startdatetime)
                    print("enddatetime", enddatetime)
                    print("current_time", current_time)

                    if startdatetime <= current_time <= enddatetime:
                        # Turn On
                        print(f"Relay {relayNumber} turned on in AutoMode")
                        break
                    else:
                        # Turn OFF
                        print(f"Relay {relayNumber} turned off in AutoMode")


turn_on_turn_off(2)

Current working directory: /home/pi/smart_plug/python_files
init called
2.6.0
2023-12-21T23:00:00
2023-12-22T00:00:00
SELECT * from automaterelay where relaynumber=2 and start_timestamp BETWEEN '2023-12-21T23:00:00' and '2023-12-22T00:00:00'
Empty DataFrame
Columns: [id, start_timestamp, end_timestamp, marketprice, unit, relaynumber, triggerstatus]
Index: []


In [2]:
## Day calculator for data download
import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import requests
from datetime import datetime, timedelta
from main_services.common_utils import common_utils 
import pytz
import time 


def get_start_and_end_of_day2(timezone='CET'):

    current_timestamp = time.time() * 1000

    timestamp = current_timestamp - (24 * 60 * 60 * 1000)

    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp / 1000)

    # Set the timezone to UTC
    dt_object_utc = pytz.timezone('UTC').localize(dt_object)

    # Convert UTC to the specified local time zone
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day
    start_of_next_day = start_of_day + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(start_of_next_day.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    
    print(f"Start of the day: {start_of_day}")
    print(f"End of the day: {end_of_day}")

    return start_timestamp, end_timestamp


get_start_and_end_of_day2()


Start of the day: 2023-12-20 00:00:00+01:00
End of the day: 2023-12-20 23:59:59.999999+01:00


(1703026800000, 1703113199999)

In [3]:
## Day calculator for awattar future data download
import sys 
sys.path.append('/home/pi/smart_plug/')
import os
import pandas as pd
import requests
from datetime import datetime, timedelta
from main_services.common_utils import common_utils 
import pytz
import time 

def get_start_and_end_of_day(timezone='CET'):
    timestamp = time.time() * 1000  # Current timestamp in milliseconds

    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp / 1000)

    # Set the timezone to UTC
    dt_object_utc = pytz.timezone('UTC').localize(dt_object)

    # Convert UTC to the specified local time zone
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day
    start_of_next_day = start_of_day + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(start_of_next_day.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    print(f"Start of the day: {start_of_day}")
    print(f"End of the day (including next start): {end_of_day}")

    return start_timestamp, end_timestamp

# Example usage:
start_timestamp, end_timestamp = get_start_and_end_of_day('CET')
print(f"Start Timestamp: {start_timestamp}")
print(f"End Timestamp: {end_timestamp}")



Start of the day: 2023-12-21 00:00:00+01:00
End of the day (including next start): 2023-12-21 23:59:59.999999+01:00
Start Timestamp: 1703113200000
End Timestamp: 1703199599999


In [4]:
## Download awattar future data
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 


from datetime import datetime
import pytz


def get_start_and_end_of_day(timezone='CET'):
    timestamp = time.time() * 1000  # Current timestamp in milliseconds

    # Convert timestamp to datetime object
    dt_object = datetime.utcfromtimestamp(timestamp / 1000)

    # Set the timezone to UTC
    dt_object_utc = pytz.timezone('UTC').localize(dt_object)

    # Convert UTC to the specified local time zone
    dt_object_local = dt_object_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day
    start_of_day = dt_object_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day = start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day
    start_of_next_day = start_of_day + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp = int(start_of_day.timestamp()) * 1000
    end_timestamp = int(start_of_next_day.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    print(f"Start of the day: {start_of_day}")
    print(f"End of the day (including next start): {end_of_day}")

    return start_timestamp, end_timestamp


def AWATTAR_FUTURE_PRICE():
    awattar_json_url = "https://api.awattar.at/v1/marketdata?start={}&end={}"
   
    timezone = 'Europe/Vienna'  # Replace with your desired timezone

    start_of_day, end_of_day = get_start_and_end_of_day(timezone)

    print(f"Awattar After Start of the day: {start_of_day}")
    print(f"Awattar After End of the day: {end_of_day}")

    # Get Awattar Data
    json_url = awattar_json_url.format(start_of_day, end_of_day)
    
    awattar_json_response = requests.get(json_url).json()
    awattar_json_response = pd.json_normalize(awattar_json_response['data'])
    awattar_json_response['start_timestamp'] = pd.to_datetime(awattar_json_response['start_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    awattar_json_response['end_timestamp'] = pd.to_datetime(awattar_json_response['end_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    print(awattar_json_response)

AWATTAR_FUTURE_PRICE()





Current working directory: /home/pi/smart_plug/python_files
Start of the day: 2023-12-21 00:00:00+01:00
End of the day (including next start): 2023-12-21 23:59:59.999999+01:00
Awattar After Start of the day: 1703113200000
Awattar After End of the day: 1703199599999
       start_timestamp       end_timestamp  marketprice     unit
0  2023-12-21 00:00:00 2023-12-21 01:00:00        34.32  Eur/MWh
1  2023-12-21 01:00:00 2023-12-21 02:00:00        30.86  Eur/MWh
2  2023-12-21 02:00:00 2023-12-21 03:00:00        25.49  Eur/MWh
3  2023-12-21 03:00:00 2023-12-21 04:00:00        12.19  Eur/MWh
4  2023-12-21 04:00:00 2023-12-21 05:00:00        10.91  Eur/MWh
5  2023-12-21 05:00:00 2023-12-21 06:00:00        39.90  Eur/MWh
6  2023-12-21 06:00:00 2023-12-21 07:00:00        55.06  Eur/MWh
7  2023-12-21 07:00:00 2023-12-21 08:00:00        76.59  Eur/MWh
8  2023-12-21 08:00:00 2023-12-21 09:00:00        78.77  Eur/MWh
9  2023-12-21 09:00:00 2023-12-21 10:00:00        72.75  Eur/MWh
10 2023-12-21 10:00

In [2]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests
import math

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)

app = Flask(__name__)

from datetime import datetime, timedelta

global conn
global cursor 




def calculateAutoModeValue(relayNumber):
    db = DatabaseManager()

    if(relayNumber == 1):
        relayPower = db.read_relaysettings_table()[0][2] # relay 1 power
    elif(relayNumber == 2):
        relayPower = db.read_relaysettings_table()[0][3] # relay 2 power
    
    last_known_times_toturn_on = db.read_automode_24hrs_before(relayNumber)
    print(last_known_times_toturn_on)

    last_known_times_toturn_on = int(last_known_times_toturn_on[0][0]) * int(relayPower)
    print(last_known_times_toturn_on)

    last_24hrs_usage = db.read_datacache_last_24hrs_consumption()[0][0] # A
    print(int(last_24hrs_usage))



    
    """
    Formula to calculate auto_mode value
    A = Last 24hrs consumption
    B = Power needed for Relay1 or Relay 2
    Result = A/B

    Example: A = 20KW; B = 5KW
        Result = 20/5 
                = 4 times (this we have to turn on and turn off the relay automatically)

    Escalation Example:
        Lets say times_turnon=6*Relay power (20kwh)
            E old = 6*20 = 120 kwh 
        If the last 24hrs consumption is greater or equal to (E Old) then multiply with escalation value
    """
    
    # Logic for Escalation --> 1.3 default value
    # Higher the demand higher the turn on time
    if(last_known_times_toturn_on >= last_24hrs_usage):
        no_of_times_to_activate_automode = last_24hrs_usage * 1.3 / int(relayPower)
        print(math.ceil(no_of_times_to_activate_automode))
        ## db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
        ##db.insert_automode(last_24hrs_usage, relayNumber,  math.ceil(no_of_times_to_activate_automode)) 

    else:
        no_of_times_to_activate_automode = int(last_24hrs_usage) / int(relayPower)
        print(math.ceil(no_of_times_to_activate_automode))
        ## db.insert_automode(last_24hrs_usage, relayNumber, round(no_of_times_to_activate_automode)) 
        ##db.insert_automode(last_24hrs_usage, relayNumber,  math.ceil(no_of_times_to_activate_automode)) 

calculateAutoModeValue(1)


Current working directory: /home/pi/smart_plug/python_files
init called
2.6.0
[(6,)]


In [1]:
import sys

sys.path.append('/home/pi/smart_plug/')
import os
import schedule
import time 
import subprocess

import sqlite3
from sqlite3 import Error

from datetime import datetime, timedelta, timezone
import json

from flask import Flask, jsonify, request, send_file, render_template

import requests
import math

from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt

from external_services.awattar_services import AwattarServices
from pi_controller.relay_controller import RelayControl
from external_services.smartmeter_services import SmartMeterServices
from database.db_manager import DatabaseManager
from main_services.common_utils import common_utils 
from main_services.auto_mode import Auto_Mode

from flask_paginate import Pagination, get_page_args

current_dir = os.getcwd()
print("Current working directory:", current_dir)


print(datetime.now())

Current working directory: /home/pi/smart_plug/python_files
2024-01-07 02:21:30.713719
